In [1]:
import sys
import time
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from tools.runners.account_state_runner import get_account_keys_online
from tools.runners.account_state_runner import report_key_files_compare
from contracts.contract_identities import DEXContractInterface

context = Context()

2024-05-23 13:58:06,310 - WARNING - Structure definition for egld_wraps not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-05-23 13:58:06,314 - WARNING - Structure definition for pairs_view not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-05-23 13:58:06,318 - WARNING - Structure definition for stakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-05-23 13:58:06,321 - WARNING - Structure definition for metastakings_boosted not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2024-05-23 13:58:06,322 - WARNING - Structure definition for position_creator not found in /home/ovi/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2024-05-23 13:58:06,731 - DEBUG - Account.sync_nonce() done: 3
2024-05-23 13:58:07,261 - DEBUG - Account.sync_nonce() done: 3
2024-05-23 13:58:07,263 - WARNING - Loaded 11 tokens instead of expected 10.
2024-05-23 13:58:07,869 - DEBUG - Account.sync_nonce() done: 3
2024-05-23 13:58:07,871 - INFO - Loaded 0 egld_wraps.
2024-05-23 13:58:07,880 - INFO - Loaded 1 locked_assets.
2024-05-23 13:58:07,882 - INFO - Loaded 1 proxies.
2024-05-23 13:58:07,883 - INFO - Loaded 0 simple_locks.
2024-05-23 13:58:07,885 - INFO - Loaded 1 simple_locks_energy.
2024-05-23 13:58:07,887 - INFO - Loaded 1 fees_collectors.
2024-05-23 13:58:07,889 - INFO - Loaded 1 unstakers.
2024-05-23 13:58:07,891 - INFO - Loaded 1 proxies_v2.
2024-05-23 13:58:07,893 - INFO - Loaded 0 router.
2024-05-23 13:58:07,895 - INFO - Loaded 1 router_v2.
2024-05-23 13:58:07,896 - INFO - Loaded 0 pairs.
2024-05-23 13:58:07,898 - INFO - Loaded 9 pairs_v2.
2024-05-23 13:58:07,899 - INFO - Loaded 0 pairs_view.
2024-05-23 13:58:07,900 - INF

In [3]:
import time

# wrap the upgrade function of each contract around a state compare procedure
def upgrade_contract_with_check(contract: DEXContractInterface, *args):
    try:
        # get the current state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/pre_contract_state.json")
    except Exception as e:
        print(f"Error getting the account state: {e}")
        # ask for user accept to continue or cancel
        if input("Continue without state? (y/n): ") != "y":
            return

    # upgrade the contract
    contract.contract_upgrade(*args)

    time.sleep(12)

    try:
        # get the new state of the account
        get_account_keys_online(contract.address, config.DEFAULT_PROXY, with_save_in="state_dump/post_contract_state.json")
        # compare the states
        report_key_files_compare("state_dump", "pre", "post", True)
    except Exception as e:
        print(f"Error getting the account state: {e}")

Energy contract setup

In [4]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [5]:
energy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

2024-05-15 17:14:12,629 - INFO - Upgrade simple lock energy contract
2024-05-15 17:14:12,631 - DEBUG - Upgrade SimpleLockEnergyContract contract
2024-05-15 17:14:13,457 - DEBUG - Upgrade arguments: []
2024-05-15 17:14:14,560 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/c4a2429b8b0277e6873c69ff619e9cab320bccd03d2e0525ef1342c1c8d9c6db


'c4a2429b8b0277e6873c69ff619e9cab320bccd03d2e0525ef1342c1c8d9c6db'

In [ ]:
upgrade_contract_with_check(energy_contract, context.deployer_account, context.network_provider.proxy, config.SIMPLE_LOCK_ENERGY_BYTECODE_PATH, [], True)

Proxy dex setup

In [6]:
from contracts.dex_proxy_contract import DexProxyContract
from utils.utils_chain import WrapperAddress as Address, hex_to_string
from utils.contract_data_fetchers import ProxyContractDataFetcher

dex_proxy_contract: DexProxyContract
dex_proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

data_fetcher = ProxyContractDataFetcher(Address(dex_proxy_contract.address), context.network_provider.proxy.url)
old_locked_token = hex_to_string(data_fetcher.get_data("getLockedTokenIds")[0])
old_factory_address = context.get_contracts(config.LOCKED_ASSETS)[0].address
print(f"old_locked_token: {old_locked_token}")
print(f"old_factory_address: {old_factory_address}")

old_locked_token: LKMEX-aab910
old_factory_address: erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th


In [7]:
dex_proxy_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [old_locked_token, old_factory_address])

2024-05-15 17:14:22,189 - INFO - upgrade DexProxyContract contract
2024-05-15 17:14:22,192 - DEBUG - Upgrade DexProxyContract contract
2024-05-15 17:14:22,469 - DEBUG - Upgrade arguments: ['LKMEX-aab910', erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th]
2024-05-15 17:14:23,597 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/105f823479337bd484c1478c946a43f81608ff8a38d730195a81fe880ea5e378


'105f823479337bd484c1478c946a43f81608ff8a38d730195a81fe880ea5e378'

In [ ]:
upgrade_contract_with_check(dex_proxy_contract, context.deployer_account, context.network_provider.proxy, config.PROXY_V2_BYTECODE_PATH, [old_locked_token, old_factory_address])

Farm contract setup

In [8]:
from contracts.farm_contract import FarmContract
farm_contract: FarmContract
farm_contracts = context.get_contracts(config.FARMS_V2)

In [ ]:
for farm_contract in farm_contracts:
    farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

In [9]:
for farm_contract in farm_contracts:
    upgrade_contract_with_check(farm_contract, context.deployer_account, context.network_provider.proxy, config.FARM_V3_BYTECODE_PATH, [], True)

2024-05-15 17:14:49,612 - INFO - Dumped the retrieved contact state in: state_dump/pre_contract_state.json
2024-05-15 17:14:49,616 - INFO - upgrade FarmContract contract
2024-05-15 17:14:49,617 - DEBUG - Arguments: []
2024-05-15 17:14:49,618 - DEBUG - Upgrade FarmContract contract
2024-05-15 17:14:49,830 - DEBUG - Upgrade arguments: []
2024-05-15 17:14:50,206 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/58328969ca7d9d91b0a888771bc27c44fa558053b59ea615f1a96dbc406749c6
2024-05-15 17:15:04,300 - INFO - Dumped the retrieved contact state in: state_dump/post_contract_state.json
2024-05-15 17:15:04,305 - INFO - 
pre_upgrade_router.json and post_upgrade_router.json are identical.
2024-05-15 17:15:04,330 - ERROR - 
pre_contract_state.json and post_contract_state.json are not identical.
2024-05-15 17:15:04,331 - WARNING - Data only in post: 6661726d5f706f736974696f6e5f6d6967726174696f6e5f6e6f6

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in farm_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Staking contract setup

In [2]:
from contracts.staking_contract import StakingContract
staking_contract: StakingContract
staking_contracts = context.get_contracts(config.STAKINGS_V2)

In [ ]:
for staking_contract in staking_contracts:
    staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

In [11]:
for staking_contract in staking_contracts:
    upgrade_contract_with_check(staking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_V3_BYTECODE_PATH, [], True)

2024-05-15 17:19:01,336 - INFO - Dumped the retrieved contact state in: state_dump/pre_contract_state.json
2024-05-15 17:19:01,337 - INFO - Upgrade staking contract
2024-05-15 17:19:01,338 - DEBUG - Upgrade StakingContract contract
2024-05-15 17:19:01,727 - DEBUG - Upgrade arguments: []
2024-05-15 17:19:02,652 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/b6f076cebc89c7993090be9656d958dfa8951dd28a25cac8ef241fdddb87c330
2024-05-15 17:19:15,407 - INFO - Dumped the retrieved contact state in: state_dump/post_contract_state.json
2024-05-15 17:19:15,410 - INFO - 
pre_upgrade_router.json and post_upgrade_router.json are identical.
2024-05-15 17:19:15,411 - ERROR - 
pre_contract_state.json and post_contract_state.json are not identical.
2024-05-15 17:19:15,412 - WARNING - Data only in post: 6661726d5f706f736974696f6e5f6d6967726174696f6e5f6e6f6e6365: 086a95
2024-05-15 17:19:15,412 - WARNING - 

In [13]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

for contract in staking_contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

2024-05-15 17:21:10,004 - DEBUG - Account.sync_nonce() done: 3591
2024-05-15 17:21:10,004 - INFO - Set boosted yield rewards percentage
2024-05-15 17:21:10,005 - DEBUG - Arguments: [6000]
2024-05-15 17:21:10,006 - DEBUG - Calling setBoostedYieldsRewardsPercentage at erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7
2024-05-15 17:21:10,006 - DEBUG - Args: [6000]
2024-05-15 17:21:10,561 - DEBUG - Contract call arguments: [6000]
2024-05-15 17:21:11,384 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/d87cdd785d9b2d38bd1fed33dc9da019d63592d6d47bf921293b713e2e297224
2024-05-15 17:21:11,386 - INFO - Set boosted yield factors
2024-05-15 17:21:11,386 - DEBUG - Arguments: [2, 1, 0, 1, 1]
2024-05-15 17:21:11,388 - DEBUG - Calling setBoostedYieldsFactors at erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7
2024-05-15 17:21:11,388 - DEBUG - Args: [2, 1, 0, 1, 1]
2024-05-15 1

In [ ]:
# resume all
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in staking_contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

Metastaking contract setup

In [14]:
from contracts.metastaking_contract import MetaStakingContract
metastaking_contract: MetaStakingContract
metastaking_contracts = context.get_contracts(config.METASTAKINGS_V2)

In [ ]:
for metastaking_contract in metastaking_contracts:
    metastaking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

In [15]:
for metastaking_contract in metastaking_contracts:
    upgrade_contract_with_check(metastaking_contract, context.deployer_account, context.network_provider.proxy, config.STAKING_PROXY_V3_BYTECODE_PATH, [], True)

2024-05-15 17:22:14,649 - INFO - Dumped the retrieved contact state in: state_dump/pre_contract_state.json
2024-05-15 17:22:14,650 - INFO - Upgrade metastaking contract
2024-05-15 17:22:14,650 - DEBUG - Upgrade MetaStakingContract contract
2024-05-15 17:22:14,890 - DEBUG - Upgrade arguments: []
2024-05-15 17:22:15,218 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/0ec39e665d05a092b6a667a669e1d57233af099d3337e0b6acd650a3c66c89f6
2024-05-15 17:22:28,384 - INFO - Dumped the retrieved contact state in: state_dump/post_contract_state.json
2024-05-15 17:22:28,385 - INFO - 
pre_upgrade_router.json and post_upgrade_router.json are identical.
2024-05-15 17:22:28,388 - INFO - 
pre_contract_state.json and post_contract_state.json are identical.
2024-05-15 17:22:28,389 - INFO - 
Found and compared 2 account state file pairs.
2024-05-15 17:22:30,772 - INFO - Dumped the retrieved contact state in: st

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)

for contract in metastaking_contracts:
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)

Fees collector contract

In [ ]:
from contracts.fees_collector_contract import FeesCollectorContract
fees_collector_contracts: FeesCollectorContract
fees_collector_contracts = context.get_contracts(config.FEES_COLLECTORS)

In [ ]:
for fees_collector_contract in fees_collector_contracts:
    upgrade_contract_with_check(fees_collector_contract, context.deployer_account, context.network_provider.proxy, config.FEES_COLLECTOR_BYTECODE_PATH, [], True)

In [5]:
from tools.runners.farm_runner import get_farm_addresses_from_chain
from contracts.farm_contract import FarmContract, FarmContractVersion

farm_addresses = get_farm_addresses_from_chain("v2")
print(farm_addresses)

for address in farm_addresses:
    farm_contract = FarmContract("", "", "", address, FarmContractVersion.V2Boosted)
    print(f'{address} : {farm_contract.get_permissions("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97", context.network_provider.proxy)}')

['erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg', 'erd1qqqqqqqqqqqqqpgqv0pz5z3fkz54nml6pkzzdruuf020gqzykp2sya7kkv', 'erd1qqqqqqqqqqqqqpgqenvn0s3ldc94q2mlkaqx4arj3zfnvnmakp2sxca2h9', 'erd1qqqqqqqqqqqqqpgqrdq6zvepdxg36rey8pmluwur43q4hcx3kp2su4yltq', 'erd1qqqqqqqqqqqqqpgq4acurmluezvmhna8tztgcrnwh0l70a2wkp2sfh6jkp', 'erd1qqqqqqqqqqqqqpgquhqfknr9dg5t0xma0zcc55dm8gwv5rpkkp2sq8f40p', 'erd1qqqqqqqqqqqqqpgq6v5ta4memvrhjs4x3gqn90c3pujc77takp2sqhxm9j', 'erd1qqqqqqqqqqqqqpgqqqckvlhd3n7ntt5w3vln4xh3tsfpj0hr2jpsxlzgcj', 'erd1qqqqqqqqqqqqqpgqcfkmz2j0e8pj0nzuk9j6mzcav0wpthq52jpsg546ch', 'erd1qqqqqqqqqqqqqpgqr9gx79xls33tnzt0a94uvnkf452fz26q2jpsau4ulw', 'erd1qqqqqqqqqqqqqpgqwcdc3ve8ww6psam9hhu5nqqjlt22nagw2jpspt9rev', 'erd1qqqqqqqqqqqqqpgqmvzrwmwkrrzug50e64kk7xerufncp47d2jpsf9m8rx', 'erd1qqqqqqqqqqqqqpgqc6fyckfxvehlwr2zvfl34dycwgtm988g2jpsrnyazn', 'erd1qqqqqqqqqqqqqpgqm78a0zmuqlm7z5qw9nseg6ecdszafljp2jpsxkm4rp', 'erd1qqqqqqqqqqqqqpgqk5xplswklltypmazcxx5mhn3jdvaxuf92jpsudss7r', 'erd1qqqq

In [6]:
from tools.runners.staking_runner import get_staking_addresses_from_chain
from contracts.staking_contract import StakingContract, StakingContractVersion

staking_addresses = get_staking_addresses_from_chain()
print(staking_addresses)

for address in staking_addresses:
    staking_contract = StakingContract("", 0, 0, 0, StakingContractVersion.V3Boosted, "", address)
    print(f'{address} : {staking_contract.get_permissions("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97", context.network_provider.proxy)}')

['erd1qqqqqqqqqqqqqpgqmqq78c5htmdnws8hm5u4suvags36eq092jpsaxv3e7', 'erd1qqqqqqqqqqqqqpgqr7kdhagkqgxvjrsk7s5333l9wwnenr9g2jps8puq33', 'erd1qqqqqqqqqqqqqpgqzps75vsk97w9nsx2cenv2r2tyxl4fl402jpsx78m9j', 'erd1qqqqqqqqqqqqqpgq45zs77q884ts6y9zj4jyqfn6ydev8ruv2jps3tteqq', 'erd1qqqqqqqqqqqqqpgqcedkmj8ezme6mtautj79ngv7fez978le2jps8jtawn', 'erd1qqqqqqqqqqqqqpgqp2wfzvkhlpkwcdxx25qzznx33345979w2jpsl3gflj', 'erd1qqqqqqqqqqqqqpgqjdlnu9ggwfc79pygn5fjjgmdm6d7vu5e2jpsw59amp', 'erd1qqqqqqqqqqqqqpgqqsx29p3fge7upkgup4mm5xsdsv4w7rh82jpsdcvdrt', 'erd1qqqqqqqqqqqqqpgqnyq8k8nfurx5rz7zxudfeeqm983uw2tvkp2shvf2ls', 'erd1qqqqqqqqqqqqqpgqt54tw2djktfa2ap8w82x37545l58ehjdkp2svarslw', 'erd1qqqqqqqqqqqqqpgq8vl7tah2auv256864pmg9885l6gs4jp3x9rs50k5x6', 'erd1qqqqqqqqqqqqqpgquttd35eztflu2apsnyewmxtsg5h9mynhx9rswhur9y', 'erd1qqqqqqqqqqqqqpgq2e4fmlhcv90wrxl2acjpcu3g2l9rpkttx9rs8pppzp', 'erd1qqqqqqqqqqqqqpgqlrl09kdjlwjdwrk8amtv5slavcundcn8x9rsjhfg4r', 'erd1qqqqqqqqqqqqqpgq32amphk0hks57az4q3wmx0ejtwjw0970x9rs3rzc5e', 'erd1qqqq